In [1]:
import sympy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display

In [2]:
def summing_sine_waves(frequencies, amplitudes, thetas, baseline=0):
    # Code modified from 
    # https://datacrayon.com/posts/signal-processing/sp-basics/summing-sine-waves/
    sinewave = np.zeros(len(time))
    for i in range(len(frequencies)):
        sinewave += amplitudes[i] * np.sin(2 * np.pi * frequencies[i] * (time) + thetas[i]) + baseline
    sinewave = sinewave/len(frequencies)

    fig = go.Figure(
        layout=dict(
            xaxis=dict(title='Time (s)'),
            yaxis=dict(title='Temperature (°C)')))

    fig.update_layout(
        autosize=False,
        height=300,
        width=900,
        margin=dict(l=20, r=20, t=20, b=20),
        xaxis = dict(
            tickmode = 'linear',
            tick0 = 0,
            dtick = 10)
        )

    fig.add_scatter(x=time, y=sinewave)
    fig.data[0].line.color = 'royalblue'
            
    return sinewave,fig


def multiple_sine_waves(frequencies, amplitudes, thetas, baseline=0, legend=False):
    # Code modified from 
    # https://datacrayon.com/posts/signal-processing/sp-basics/summing-sine-waves/
    fig = make_subplots(
        rows=len(frequencies), 
        cols=1, 
        shared_xaxes=True,
        shared_yaxes='all')

    fig.update_layout(
        height=300,
        width=900,
        margin=dict(l=20, r=20, t=20, b=20),
        showlegend=legend)

    sinewaves = np.zeros(len(time))
    for i in range(len(frequencies)):
        sinewaves = amplitudes[i] * np.sin(2 * np.pi * frequencies[i] * (time) + thetas[i]) + baseline
        fig.add_scatter(
        x=time,
        y=sinewaves,
        row=i+1, col=1,
        name=f'wave {i+1}')
    fig.show()

    return fig


In [3]:
def symbolic_functions(frequencies, amplitudes, thetas, baseline=0, display_functions=True, export_f=False):
    '''The same as above but with sympy
    Returns:
        f: lambdify function
        f_dot: lambdify derivative of f
    '''
    
    x = sympy.symbols('x')
    wavelines = sympy.S.Zero
    for i in range(len(frequencies)):
        wavelines += amplitudes[i] * sympy.sin(2 * sympy.pi * frequencies[i] * x + thetas[i]) + baseline
    f = sympy.nsimplify(wavelines / len(frequencies)) # nsimplify to simplify fractions
    f_dot = sympy.diff(f, x)
    
    if display_functions: # before lambdify
        print("f(x) and f'(x) are:")
        display(f,f_dot)
        
    # export mathematical function
    if export_f:
        with open('stimuli_f.txt', 'w') as f_file:
            f_file.write(str(f))
        with open('stimuli_f_dot.txt', 'w') as f_dot_file:
            f_dot_file.write(str(f_dot))
            
    # lamdify functions
    f = sympy.lambdify(x, f, 'numpy')
    f_dot = sympy.lambdify(x, f_dot, 'numpy')
        
    return f, f_dot


def stimuli_extra(f,f_dot,time, s_RoC=0.5):
    '''For plotly graphing of f(x), f'(x) and labels
    Returns:
        labels: 0 for cooling, 1 for heating
        labels_alt: 0 for cooling, 1 for heating, 2 for RoC < s_RoC
    '''
    fig = go.Figure(
    layout=dict(
        xaxis=dict(title='Time (s)'),
        yaxis=dict(title='Temperature (°C) \ RoC (°C/s)')))

    fig.update_layout(
    autosize=False,
    width=900,
    margin=dict(l=20, r=20, t=20, b=20),
    xaxis = dict(
        tickmode= 'linear',
        tick0 = 0,
        dtick = 10)
    )
    
    # 0 for cooling, 1 for heating
    labels = (f_dot(time)>0).astype(int)
    # alternative: 0 for cooling, 1 for heating, 2 for RoC < s_RoC
    labels_alt = np.where(
        np.abs(f_dot(time))>s_RoC, 
        labels, 2)

    func = [f(time),f_dot(time),labels]
    func_names = "f(x)","f'(x)","Label"
    colors = 'royalblue', 'skyblue', 'springgreen'
    for idx, i in enumerate(func):
        fig.add_scatter(x=time, y=i, name=func_names[idx])
        fig.data[idx].line.color = colors[idx]
    
    fig.add_scatter(x=time, y=labels_alt, name="Label (alt)", visible="legendonly")

    fig.show()
    
    return labels, labels_alt, fig

def cooling_segments(labels):
    '''Displays the number and length of cooling segments'''
    change = np.concatenate([
        np.array([0]), # sign cannot change with first value
        np.diff(labels>0)*1], axis=0)

    # returns a list of indices where the conditions have been met
    # (np.where()[0] is tricky)
    change_idx = np.where(change==1)[0]
    
    match labels[0]: # label we started with (cooling or heating)
        # in case 0 the first change_idx starts the first heating segment, 
        # that is why we start with i=1::2 and not i=0::2
        # (we don't prepend / append any values from np.diff here)
        case 0:
            segments = {
                idx:np.diff(change_idx)[i::2] for idx, i in enumerate(list(range(2))[::-1])
                }
        case 1:
            segments = {
                i:np.diff(change_idx)[i::2] for i in list(range(2))
                }

    # in seconds; only 1 column because jagged arrays can appear
    display(pd.DataFrame(
        {"Cooling segments [sec]":segments[0]/sample_rate}
        ).describe().applymap('{:,.2f}'.format))
    
    return change, change_idx, segments


In [4]:
# Parameters
sample_rate = 10
start_time = 0
end_time = 60*5

time = np.arange(
    start_time,
    end_time,
    1/sample_rate)

baseline = 32

period_lengths = np.array([18,60]) # in seconds; 0,3/1 gives a good ratio
frequencies = 1./period_lengths
amplitudes = [4]*len(frequencies)
thetas = [0*np.pi]*len(frequencies) 
# for individual phases use thetas = np.array([0,0])*np.pi

# Sine Waves


In [5]:
# Plot stimuli
wave, fig = summing_sine_waves(frequencies, amplitudes, thetas, baseline=baseline)
fig.add_hline(y=baseline)
fig.show()

_ = multiple_sine_waves(frequencies, amplitudes, thetas, baseline=baseline)

In [6]:
# Mathematical representations
f, f_dot = symbolic_functions(frequencies, amplitudes, thetas, baseline=baseline, display_functions=True, export_f=True)

# Plot derivative & labels
labels, labels_alt, fig_extra = stimuli_extra(f, f_dot, time, s_RoC=0.3)

# Analyze cooling segments
_ = cooling_segments(labels)

f(x) and f'(x) are:


2*sin(pi*x/30) + 2*sin(pi*x/9) + 32

pi*cos(pi*x/30)/15 + 2*pi*cos(pi*x/9)/9

,Cooling segments [sec]
count,16.00
mean,9.08
std,1.12
min,7.50
25%,8.10
50%,9.10
75%,10.00
max,10.40


In [7]:
# TODO
# - prepend / append values from np.diff to get the first and last segment
# - refactor cooling_segements with np.where if possible -> would be so much cleaner
# - add labels for - and + RoC, for no pain and high pain (dont overinterpret, beware of psychological expectation effects)
#
# - unit test for max length of segments (should never be higer than 35 sec (?)), using nbdev
# - fig.show("png") when script is finalized for github / save fig as png